<a href="https://colab.research.google.com/github/NagmanovRay/Data-Projects/blob/main/Cost_of_Living.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pycountry-convert

     |████████████████████████████████| 10.1 MB 4.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 207 kB 58.1 MB/s 
     |████████████████████████████████| 297 kB 48.2 MB/s 
     |████████████████████████████████| 297 kB 41.1 MB/s 
     |████████████████████████████████| 297 kB 58.7 MB/s 
     |████████████████████████████████| 296 kB 59.5 MB/s 
     |████████████████████████████████| 296 kB 59.9 MB/s 
     |████████████████████████████████| 280 kB 37.7 MB/s 
     |████████████████████████████████| 280 kB 44.3 MB/s 
     |████████████████████████████████| 280 kB 72.0 MB/s 
     |████████████████████████████████| 280 kB 63.4 MB/s 
     |████████████████████████████████| 279 kB 73.9 MB/s 
     |████████████████████████████████| 279 kB 67.5 MB/s 
     |████████████████████████████████| 272 kB 50.0 MB/s 
     |████████████████████████████████| 272 kB 59.9 MB/s 
    

In [2]:
import numpy as np #linear alg
import pandas as pd #data processing, CSV fil I/O
import plotly.express as px #charts plot
import plotly.graph_objects as go #plot customizstion
import pycountry_convert as pc #Country -> continent
import statsmodels #Statistics odels


In [4]:
pip install kaggle

In [6]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [9]:
! kaggle datasets download ankanhore545/cost-of-living-index-2022


  0% 0.00/3.71k [00:00<?, ?B/s]
100% 3.71k/3.71k [00:00<00:00, 3.36MB/s]


In [15]:
from google.colab import files
uploaded = files.upload()

Saving Cost_of_Living_Index_2022.csv to Cost_of_Living_Index_2022.csv


In [22]:
import io
cost = pd.read_csv(io.BytesIO(uploaded['Cost_of_Living_Index_2022.csv']),header=1)

In [23]:
cost.head()


,Rank,Country,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,1,Afghanistan,20.37,2.72,12.09,14.92,12.41,23.04
1,2,Albania,35.50,8.47,22.83,29.32,25.82,30.19
2,3,Algeria,26.87,4.59,16.43,28.82,14.48,24.63
3,4,Argentina,34.69,7.71,22.04,28.17,33.32,30.72
4,5,Armenia,33.89,11.61,23.45,27.59,30.55,28.86


In [24]:
cost.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Rank                            139 non-null    int64  
 1   Country                         139 non-null    object 
 2   Cost of Living Index            139 non-null    float64
 3   Rent Index                      139 non-null    float64
 4   Cost of Living Plus Rent Index  139 non-null    float64
 5   Groceries Index                 139 non-null    float64
 6   Restaurant Price Index          139 non-null    float64
 7   Local Purchasing Power Index    139 non-null    float64
dtypes: float64(6), int64(1), object(1)
memory usage: 8.8+ KB


In [25]:
cost.describe()

,Rank,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
count,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000
mean,70.000000,50.188633,19.291511,35.705324,46.637842,43.444892,46.426259
std,40.269923,20.860222,15.317726,17.542523,20.952229,24.885969,26.921840
min,1.000000,19.920000,2.720000,12.090000,14.920000,12.410000,1.450000
25%,35.500000,34.715000,9.515000,23.310000,31.200000,25.290000,27.040000
50%,70.000000,44.680000,13.930000,31.260000,40.220000,34.560000,37.220000
75%,104.500000,62.580000,25.070000,44.865000,56.590000,54.925000,66.105000
max,139.000000,146.040000,98.580000,123.800000,148.660000,159.170000,118.440000


In [26]:
replace_values = {'Bosnia And Herzegovina': 'Bosnia and Herzegovina',  'Trinidad And Tobago': 'Trinidad and Tobago'}  
cost['Country'] = cost['Country'].replace(replace_values)

In [27]:
def country_to_continent(country_name):
    if country_name != 'Kosovo (Disputed Territory)':
        country_alpha2 = pc.country_name_to_country_alpha2(country_name)
        country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    else:
        country_continent_name = 'Europe'
    return country_continent_name

cost['Continent'] = cost['Country'].apply(lambda x: country_to_continent(x))

In [29]:
def assign_color(continent_column, color_map):
    color_column = ''
    for continent, color in color_map.items():
        if continent_column == continent:
            color_column = color
    return color_column

In [30]:
colors_and_continents = {'Asia': '#333ED4', 
                         'Europe': '#FD0100', 
                         'Africa': '#2FA236',
                         'South America': '#EEDE04', 
                         'Oceania': '#A0D636',
                         'North America': '#F76915'}
cost['Color'] = cost['Continent'].apply(lambda x: assign_color(x, colors_and_continents))
cost

,Rank,Country,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,Continent,Color
0,1,Afghanistan,20.37,2.72,12.09,14.92,12.41,23.04,Asia,#333ED4
1,2,Albania,35.50,8.47,22.83,29.32,25.82,30.19,Europe,#FD0100
2,3,Algeria,26.87,4.59,16.43,28.82,14.48,24.63,Africa,#2FA236
3,4,Argentina,34.69,7.71,22.04,28.17,33.32,30.72,South America,#EEDE04
4,5,Armenia,33.89,11.61,23.45,27.59,30.55,28.86,Asia,#333ED4
...,...,...,...,...,...,...,...,...,...,...
134,135,Venezuela,43.20,9.99,27.63,37.34,46.23,15.41,South America,#EEDE04
135,136,Vietnam,37.48,13.22,26.11,38.70,19.98,29.38,Asia,#333ED4
136,137,Yemen,53.14,6.71,31.38,67.12,24.74,14.41,Asia,#333ED4
137,138,Zambia,33.04,9.26,21.90,31.53,22.46,40.33,Africa,#2FA236


In [40]:
fig = go.Figure()
for continent, color in colors_and_continents.items():
  subset = cost[cost['Continent']==continent]
  fig.add_trace(go.Scatter(
                x=subset['Cost of Living Index'], y=subset['Rent Index'],
                name=continent,
                text=subset['Country'],
                mode='markers',
                marker=dict(color=color, size=10, opacity=0.75)))
  
  fig.update_layout(title="Cost of Living Index vs Rent Index Bubble Chart",
                  title_x=0.45,
                  xaxis_title="Cost of Living Index",
                  yaxis_title="Rent Index",
                  showlegend=True)

fig.show()

In [41]:
fig = px.scatter(cost, x=cost['Cost of Living Index'], y = cost['Rent Index'], 
                 color=cost['Continent'],color_discrete_map=colors_and_continents,
                 trendline='ols')

fig.update_layout(title="Cost of Living Index vs Rent Index: Trendlines",
                  title_x=0.45,
                  xaxis_title="Cost of Living Index",
                  yaxis_title="Rent Index")
fig.show()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [43]:
fig = go.Figure()
for continent, color in colors_and_continents.items():
    subset = cost[cost['Continent'] == continent]
    fig.add_trace(go.Scatter(
                x=subset['Groceries Index'], y = subset['Restaurant Price Index'],
                name=continent,
                text=subset['Country'],
                mode='markers',
                marker = dict(color=color, size=10, opacity=0.75)))
    
fig.update_layout(title="Groceries Index vs Restaurant Price Index Bubble Chart",
                  title_x=0.45,
                  xaxis_title="Groceries Index",
                  yaxis_title="Restaurant Price Index",
                  showlegend=True)

fig.show()

In [44]:
fig = px.scatter(cost, x=cost['Groceries Index'], y = cost['Restaurant Price Index'], 
                 color=cost['Continent'],color_discrete_map=colors_and_continents, 
                 trendline="ols")

fig.update_layout(title="Groceries Index vs Restaurant Price Index: Trendlines",
                  title_x=0.45,
                  xaxis_title="Groceries Index",
                  yaxis_title="Restaurant Price Index")
fig.show()

In [46]:
fig = go.Figure()
for continent, color in colors_and_continents.items():
    subset = cost[cost['Continent'] == continent]
    fig.add_trace(go.Scatter(
                x=subset['Cost of Living Plus Rent Index'], y = subset['Local Purchasing Power Index'],
                name=continent,
                text=subset['Country'],
                mode='markers',
                marker = dict(color=color, size=10, opacity=0.75)))
    
fig.update_layout(title="Cost of Living Plus Rent Index vs Local Purchasing Power Bubble Chart",
                  title_x=0.45,
                  xaxis_title="Cost of Living Plus Rent Index",
                  yaxis_title="Locsl Purchasing Power Index",
                  showlegend=True)

fig.show()

In [47]:
fig = px.scatter(cost, x=cost['Cost of Living Plus Rent Index'], y = cost['Local Purchasing Power Index'],
                 color=cost['Continent'],color_discrete_map=colors_and_continents, 
                 trendline='ols')

fig.update_layout(title="Cost of Living Plus Rent Index vs Local Purchasing Power Index: Trendlines",
                  title_x=0.45,
                  xaxis_title='Cost of Living Plus Rent Index',
                  yaxis_title="Local Purchasing Power Index")
fig.show()

In [50]:
cost_by_continent = cost.groupby(['Continent'])["Cost of Living Index"].agg('mean').to_frame(name = 'Mean Cost of Living Index').reset_index()
rent_by_continent = cost.groupby(['Continent'])["Rent Index"].agg('mean').to_frame(name = 'Mean Rent Index').reset_index()
cost_and_rent = cost_by_continent
cost_and_rent['Mean Rent Index'] = rent_by_continent['Mean Rent Index']
cost_and_rent['Color'] = cost_and_rent['Continent'].apply(lambda x: assign_color(x, colors_and_continents))
cost_and_rent.round(decimals=0)

,Continent,Mean Cost of Living Index,Mean Rent Index,Color
0,Africa,39.0,13.0,#2FA236
1,Asia,46.0,19.0,#333ED4
2,Europe,58.0,23.0,#FD0100
3,North America,60.0,23.0,#F76915
4,Oceania,65.0,30.0,#A0D636
5,South America,38.0,10.0,#EEDE04


In [57]:
fig = go.Figure(data=[
                      go.Bar(name='Mean cost of Living Index',
                             x=cost_and_rent['Continent'],
                             y=cost_and_rent['Mean Cost of Living Index'],
                             marker_color = cost_and_rent['Color'],
                             marker_opacity=1,
                             text='CoL'),
                      
                      go.Bar(name='Mean Rent Index',
           x=cost_and_rent['Continent'], 
           y=cost_and_rent['Mean Rent Index'],
           marker_color=cost_and_rent['Color'], 
           marker_opacity=0.5,
           text='Rent')
])

fig.show()